# Battle of the Neighborhoods - New Dispensary
## Jordan Meyer https://github.com/jmdatasci

# Introduction

With the legal landscape rapidly changing, regulation and decriminalization of Marijuana is significantly increasing.  A substance that was once considered dangerous through urban legends and propaganda has been debunked and is being touted for a combination of its medicinal benefits, increased safety vs alternative substances, plant by-products and more.  In the changing environment, it is the perfect time to invest in this growing $16 Billion industry.  
  
The goal of this project is to determine the optimal location for opening a new dispensary in cities in which marijuana is recreationally legal.  The initial location and underlying data may then be used to find a storefront, establish a supply chain, build marketing campaign and develop a fiscal business plan to get investor contribution.

# Data 

## To determine the best location for opening a number of factors will be looked at:

#### 2019 State Laws
###### https://disa.com/map-of-marijuana-legality-by-state

#### 2019 Local Venue Data
###### https://foursquare.com

#### 2017-2018 Population and Crime Statistics  
###### https://ucr.fbi.gov/crime-in-the-u.s/2018/preliminary-report/tables/table-4/table-4.xls/view  

#### 2019 State Tax Rate
###### https://taxfoundation.org/sales-tax-rates-2019/ 

#### List of US States
###### https://simple.wikipedia.org/wiki/List_of_U.S._states

#### 2018 Cost of Living Index 
##### https://www.kaggle.com/andytran11996/cost-of-living/version/3

- First, the legality will be assessed for each state and city.  
- The legality will be cross referenced against a US-population database to find the most densely populated cities with population > 100,000 people.
- Locations of current dispensaries within 1 mile (1.609 km) will be checked.  
 - If a dispensary exists, the location will be ruled out.  
- Another factor that will be assessed is the demographic of the area.  
 - Cities or neighborhoods with children and families will not be looked at in order to keep them family-friendly.  
- Since the goal of this project is to open a recreational dispensary 
 - I will be looking in areas with active nightlife and adult activities 
 - I will be looking for places with nearby fast-food, pizza restaurants and convenience stores.  Establishments are often frequented by target clientele
- I will be verifying crime rates in cities and aim for lower property crime rate locations
 - Many dispensaries have been robbed since installation due to the nature of the product.  Try to minimize this risk as much as possible.

  
#### Stipulations and Assumptions

1. The following states did not have crime data available and were not included in this assessment.  This was not a major problem since none of these states had recreational marijuana legal except for Vermont.   
    - Delaware
    - Maine
    - Mississippi
    - Vermont*  
    - West Virginia 
    - Wyoming 
 
1. Although a legalized area, District of Columbia was not considered in this report since it is not a state and therefore did not have state data available. 
1. Only property crime was considered.  Violent crimes, although bad, were not taken into account since the risk posed in this endeavor is robbery, burglary or larceny.   
  
1. Since this project is created to get investors and not based on a specific region, only cities with 100,000 residents or more were considered, additionally, anywhere in the United States was considered to be Viable.
 
 


## Methodology 

In order to gather the appropriate data, signficant data cleansing was performed to get information into a usable format, remove bad data, exclude cities that are not in scope and generate a streamlined dataset.

I began with consolidating all data based on state/city.  To do this I took the tax information which had abbreviated state names and combined them with a list of the 50 states from wikipedia.  Once I had the taxes and state names available in its own dataframe I was able to combine it with the crime statistics dataframe.  I also got geospatial data from GeoPy using the city and state names to add to the city database.  Finally, I calculated a crime-index which is a ratio of property crime to population for each city to complete the database.  

Once the database was established, I used the Geo-coordinate data to query the Foursquare server API and gather local venue data for each city.  Cities that had dispensaries were dropped from the dataset.  The venue data was then transformed with one-hot encoding and grouped along venue frequency by the counts in each city.  This data was then used to cluster the cities into 5 different clusters based on what types of venue and overall culture is present in the area.  

I started performing exploratory analysis by creating a map of all of the cities that were in my dataset.  After clustering was performed, a new map was generated color coding the clusters on the map.  Finally, once the target cluster was identified, the crime rate and sales tax were cross referenced to determine the top choice of city to open the dispensary.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
import folium # map rendering library
from sklearn.cluster import KMeans # import k-means from clustering stage
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from bs4 import BeautifulSoup
#!conda install beautifulsoup4 --yes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')


Libraries imported.


## Get updated legalization data

In [2]:
url = "disa.com/map-of-marijuana-legality-by-state"
r  = requests.get("http://" +url)
data = r.text
soup = BeautifulSoup(data)

In [3]:
text = []
legalized = []
medical = []
i=0
for table in soup.find_all('td'):
    text.append([i, table])
    i+=1

for j in range(0,len(text)-1):
    if str(text[j+1][1]) == "<td>Fully Legal</td>":
        s = str(text[j][1])
        state = s[4:-5]
        legalized.append(state.upper())
    elif str(text[j+1][1]) == "<td>Mixed</td>":
        s = str(text[j][1])
        state = s[4:-5]
        medical.append(state.upper())

In [4]:
legalized

['ALASKA',
 'CALIFORNIA',
 'COLORADO',
 'DISTRICT OF COLUMBIA',
 'MAINE',
 'MASSACHUSETTS',
 'MICHIGAN',
 'NEVADA',
 'OREGON',
 'VERMONT',
 'WASHINGTON']

## State Sales Tax 

In [5]:
url = "taxfoundation.org/sales-tax-rates-2019/"
r  = requests.get("http://" +url)
data = r.text
soup = BeautifulSoup(data)

In [6]:
text = []

for table in soup.find_all('td'):
    text.append(table)
text.pop(0)

state = []
tax = []

for i in range(0,len(text)):
    if i % 7 == 0:
        s = str(text[i])
        t = str(text[i+1])
        state.append(s[4:-5])
        tax.append(t[4:-5])



In [7]:
for j in range(0,len(tax)):
    try :
        tax[j] = float(tax[j].strip('%'))/100
    except :
        tax[j] = float(0)

In [8]:
for k in range(0,len(tax)):
    try :
        state[k] = state[k].split(' ', 1)[0]
        state[k] = state[k].split('\xa0', 1)[0]
    except :
        next 

## Correct State Names

In [9]:
url = "simple.wikipedia.org/wiki/List_of_U.S._states"
r  = requests.get("http://" +url)
data = r.text
soup = BeautifulSoup(data)

In [10]:
text = []

for table in soup.find_all('area'):
    text.append(str(table))
statelist =[]

for k in range(0,len(text)):
    try :
        text[k] = text[k][11:].split('"', 1)[0]
        text[k] = text[k].split(' CA', 1)[0]
        statelist.append(text[k].upper())
    except :
        next 

In [11]:
statelist.sort()
statelist = list(dict.fromkeys(statelist))

In [12]:
taxes = pd.DataFrame([statelist,tax[0:50]]).transpose()

In [13]:
taxes.rename(columns={0:'State',1:'Tax'},inplace=True)
taxes.head()

,State,Tax
0,ALABAMA,0.04
1,ALASKA,0
2,ARIZONA,0.056
3,ARKANSAS,0.065
4,CALIFORNIA,0.0725


### Get cost of living data

In [14]:
costol = pd.read_csv(r'cost-of-living-2018.csv')

In [15]:
costol.head()

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,NaN,"Hamilton, Bermuda",145.43,110.87,128.76,143.47,158.75,112.26
1,NaN,"Zurich, Switzerland",141.25,66.14,105.03,149.86,135.76,142.70
2,NaN,"Geneva, Switzerland",134.83,71.70,104.38,138.98,129.74,130.96
3,NaN,"Basel, Switzerland",130.68,49.68,91.61,127.54,127.22,139.01
4,NaN,"Bern, Switzerland",128.03,43.57,87.30,132.70,119.48,112.71


In [16]:
coldb = pd.DataFrame()

In [17]:
coldb.head()

""


In [18]:
for i in range(0,len(costol)):
    coldb.loc[i,'City'] = costol['City'][i].split(', ')[0].upper()
    coldb.loc[i,'CLI'] = costol['Cost of Living Index'][i]

In [19]:
coldb.head()

,City,CLI
0,HAMILTON,145.43
1,ZURICH,141.25
2,GENEVA,134.83
3,BASEL,130.68
4,BERN,128.03


### Generate Legalization Map

In [20]:
!wget --quiet https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json?_sm_au_=iHV3MMrsT47Sts2R -O states.json

In [21]:
legal_map = folium.Map(location=[39.83, -98.58], zoom_start=4, tiles='Mapbox Bright')

In [22]:
legaldf = pd.DataFrame(legalized)

In [23]:
medicaldf = pd.DataFrame(medical)

In [24]:
medicaldf.head()

,0
0,ARIZONA
1,ARKANSAS
2,CONNECTICUT
3,DELAWARE
4,FLORIDA


In [25]:
legaldf['Legal'] = 2
medicaldf['Legal'] = 1

In [26]:
legaldf.rename(columns={0:'State'},inplace=True)
medicaldf.rename(columns={0:'State'},inplace=True)

In [27]:
legdb = pd.concat([legaldf,medicaldf])

In [28]:
legdb.head()

,State,Legal
0,ALASKA,2
1,CALIFORNIA,2
2,COLORADO,2
3,DISTRICT OF COLUMBIA,2
4,MAINE,2


In [29]:
newdb = taxes.merge(legdb, on="State",how="outer")

In [30]:
newdb = newdb.fillna(0)

In [31]:
newdb["Legal"] = newdb["Legal"].astype(int)
for i in range(0,len(newdb["State"])):
    newdb.loc[i,"State"]= newdb["State"][i].capitalize()

In [32]:
newdb.head()

,State,Tax,Legal
0,Alabama,0.0400,0
1,Alaska,0.0000,2
2,Arizona,0.0560,1
3,Arkansas,0.0650,1
4,California,0.0725,2


In [33]:
statemap = r'states.json' # geojson file

# create a plain world map
legal_map = folium.Map(location=[39.83, -98.58], zoom_start=4, tiles='Mapbox Bright')
legal_map.choropleth(
    geo_data=statemap,
    data=newdb,
    columns=["State","Legal"],
    key_on='feature.properties.name',
    fill_color="GnBu",
    fill_opacity=0.7, 
    line_opacity=0.2,
    threshold_scale=[0.001,1.001,2.001],
    legend_name='Legalization in the US 2019 - Blue: Fully Legal, Green: Medicinal only'

)

# display map
legal_map

## Import Crime Statistics 

In [34]:
crime = pd.read_csv(r'crime_stats.csv',skiprows=4)
crime = crime[:-9]

In [35]:
taxes.head()

,State,Tax
0,ALABAMA,0.04
1,ALASKA,0
2,ARIZONA,0.056
3,ARKANSAS,0.065
4,CALIFORNIA,0.0725


In [36]:
states = list(crime['State'].unique())
del states[1]

In [37]:
crime = crime.merge(taxes, on='State', how='left')

In [38]:
for i in range (1,len(crime)):
    if i%2 != 0:
        crime.loc[i, 'City'] = crime.loc[i-1, 'City']
    if pd.isnull(crime.loc[i,'State']):
        crime.loc[i, 'State'] = crime.loc[i-1, 'State']
        crime.loc[i, 'Tax'] = crime.loc[i-1, 'Tax']

In [39]:
crime.rename(columns={"Population1":"Population","Property \ncrime":"Property Crime", "Larceny-\ntheft":"Theft"}, inplace=True)
crime.drop(columns=["Unnamed: 2","Violent \ncrime","Murder","Rape2", "Aggravated \nassault","Motor \nvehicle \ntheft","Arson3","Unnamed: 14","Burglary","Theft","Robbery"],inplace=True)
crime = crime[:-9]

In [40]:
crime = crime.fillna(0)

In [41]:
crime["Population"] = crime["Population"].str.replace(",","").astype(float)
crime["Property Crime"] = crime["Property Crime"].str.replace(",","").astype(float)

In [42]:
crime = crime[crime['Population'] != 0]

In [43]:
crime['Crime Index'] = crime['Property Crime']/crime['Population']

In [44]:
crime.dropna(axis=0,inplace=True)

In [45]:
crime = crime.reset_index(drop=True)
crime.head()

,State,City,Population,Property Crime,Tax,Crime Index
0,ALABAMA,BIRMINGHAM,212178.0,6472.0,0.04,0.030503
1,ALABAMA,MOBILE4,248431.0,6493.0,0.04,0.026136
2,ALABAMA,MONTGOMERY,199099.0,4246.0,0.04,0.021326
3,ALABAMA,TUSCALOOSA,101124.0,1953.0,0.04,0.019313
4,ALASKA,ANCHORAGE,296188.0,7708.0,0.00,0.026024


In [46]:
coldb.head()

,City,CLI
0,HAMILTON,145.43
1,ZURICH,141.25
2,GENEVA,134.83
3,BASEL,130.68
4,BERN,128.03


In [47]:
crime = crime.merge(coldb, on='City', how='left')

In [48]:
try:
    crime.drop(['Country'],inplace=True,axis=1)
except:
    next

In [49]:
crime=crime.fillna(0)

In [50]:
crime.head()

,State,City,Population,Property Crime,Tax,Crime Index,CLI
0,ALABAMA,BIRMINGHAM,212178.0,6472.0,0.04,0.030503,74.64
1,ALABAMA,BIRMINGHAM,212178.0,6472.0,0.04,0.030503,71.60
2,ALABAMA,MOBILE4,248431.0,6493.0,0.04,0.026136,0.00
3,ALABAMA,MONTGOMERY,199099.0,4246.0,0.04,0.021326,0.00
4,ALABAMA,TUSCALOOSA,101124.0,1953.0,0.04,0.019313,0.00


In [51]:
try:
    for i in range(0,len(crime)):
        if crime['State'][i] not in legalized:
            crime.drop([i],axis=0, inplace=True)
except:
    next

In [52]:
crime.head()

,State,City,Population,Property Crime,Tax,Crime Index,CLI
5,ALASKA,ANCHORAGE,296188.0,7708.0,0.0000,0.026024,94.99
16,CALIFORNIA,ANAHEIM,353400.0,4629.0,0.0725,0.013098,0.00
17,CALIFORNIA,ANTIOCH,112252.0,1873.0,0.0725,0.016686,0.00
18,CALIFORNIA,BAKERSFIELD,381154.0,7570.0,0.0725,0.019861,69.20
19,CALIFORNIA,BERKELEY,122687.0,2827.0,0.0725,0.023042,87.82


### Find the city with lowest  Property Crime City to open a dispensary in each state

In [53]:
crime = crime[crime.City != 'RIALTO5']
crime = crime[crime.City != 'LAS VEGAS METROPOLITAN POLICE DEPARTMENT']

In [54]:
cities = pd.concat([crime['State'],crime['City'],crime['Tax'],crime['Crime Index'],crime['CLI']],axis=1)

In [55]:
cities.reset_index(inplace=True,drop=True)

In [56]:
cities.head()

,State,City,Tax,Crime Index,CLI
0,ALASKA,ANCHORAGE,0.0000,0.026024,94.99
1,CALIFORNIA,ANAHEIM,0.0725,0.013098,0.00
2,CALIFORNIA,ANTIOCH,0.0725,0.016686,0.00
3,CALIFORNIA,BAKERSFIELD,0.0725,0.019861,69.20
4,CALIFORNIA,BERKELEY,0.0725,0.023042,87.82


### Get Geospatial Data using GeoPy for City, State

In [57]:
lat = []
lon = []
for i in range(0,len(cities)):
    address = '{}, {}'.format(cities['City'][i],cities['State'][i])
    geolocator = Nominatim(user_agent='capstone')
    location = geolocator.geocode(address)
    lat.append(location.latitude)
    lon.append(location.longitude)

In [62]:
city2 = pd.concat([cities,pd.Series(lat),pd.Series(lon)],axis=1)

In [63]:
city2.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)

In [164]:
city2.head()

,State,City,Tax,Crime Index,CLI,Latitude,Longitude
0,ALASKA,ANCHORAGE,0.0000,0.026024,94.99,61.216313,-149.894852
1,CALIFORNIA,ANAHEIM,0.0725,0.013098,0.00,33.834752,-117.911732
2,CALIFORNIA,ANTIOCH,0.0725,0.016686,0.00,38.004921,-121.805789
3,CALIFORNIA,BAKERSFIELD,0.0725,0.019861,69.20,35.373871,-119.019464
4,CALIFORNIA,BERKELEY,0.0725,0.023042,87.82,37.870839,-122.272864


### Create map of all viable US cities with population over 100,000 and legalized Recreational Marijuana

In [65]:
# create map of Cities using latitude and longitude values
cities_map = folium.Map(location=[39.83, -98.58], zoom_start=4)

# add markers to map
for lat, lng, city, state in zip(city2['Latitude'], city2['Longitude'], city2['City'], city2['State']):
    label = '{}, {}'.format(state, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#228B22',
        fill_opacity=0.7,
        parse_html=False).add_to(cities_map)  
    
cities_map

### Get local venue data for each city from Foursquare API

In [66]:
CLIENT_ID = 'K5HRW4OC5J4EZS14DH2OKQBNZV5GNXIOZKY03QXJ4RDCPTUD' # your Foursquare ID
CLIENT_SECRET = '4ZWZFMI0VAL01GTTVLUJSJOL0YPARUAQB225RMYILABEIQ0Q' # your Foursquare Secret
VERSION = '20190420'
limit = 100
radius = 1609
latitude = 1
longitude = 1
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentials:
CLIENT_ID: K5HRW4OC5J4EZS14DH2OKQBNZV5GNXIOZKY03QXJ4RDCPTUD
CLIENT_SECRET:4ZWZFMI0VAL01GTTVLUJSJOL0YPARUAQB225RMYILABEIQ0Q


In [81]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    radius,
    limit)


In [82]:
results = requests.get(url).json()

In [83]:
def getNearbyVenues(states, names, latitudes, longitudes, radius=1609):
    
    venues_list=[]
    for state, name, lat, lng in zip(states, names, latitudes, longitudes):
                  
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            state,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['State',
                             'City',
                              'City Latitude', 
                              'City Longitude', 
                              'Venue', 
                              'Venue Latitude', 
                              'Venue Longitude', 
                              'Venue Category']
    
    return(nearby_venues)

In [84]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [85]:
city2.head()

,State,City,Tax,Crime Index,CLI,Latitude,Longitude
0,ALASKA,ANCHORAGE,0.0000,0.026024,94.99,61.216313,-149.894852
1,CALIFORNIA,ANAHEIM,0.0725,0.013098,0.00,33.834752,-117.911732
2,CALIFORNIA,ANTIOCH,0.0725,0.016686,0.00,38.004921,-121.805789
3,CALIFORNIA,BAKERSFIELD,0.0725,0.019861,69.20,35.373871,-119.019464
4,CALIFORNIA,BERKELEY,0.0725,0.023042,87.82,37.870839,-122.272864


In [86]:
city_venues = getNearbyVenues(states=city2['State'],
                                   names= city2['City'],
                                   latitudes = city2['Latitude'],
                                   longitudes = city2['Longitude'],
                                   radius=1609
                                  )

In [87]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


In [88]:
city_venues.head()

,State,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALASKA,ANCHORAGE,61.216313,-149.894852,Glacier BrewHouse,61.217719,-149.896839,Brewery
1,ALASKA,ANCHORAGE,61.216313,-149.894852,Humpy's Great Alaskan Alehouse,61.216427,-149.894146,Bar
2,ALASKA,ANCHORAGE,61.216313,-149.894852,Crow's Nest,61.217838,-149.899718,Seafood Restaurant
3,ALASKA,ANCHORAGE,61.216313,-149.894852,49th State Brewing,61.219736,-149.895975,Brewery
4,ALASKA,ANCHORAGE,61.216313,-149.894852,Apple Anchorage 5th Avenue Mall,61.217140,-149.888671,Electronics Store


### Prepare the venue data to perform K-means clustering

In [89]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['State'] = city_venues['State'] 
city_onehot['City'] = city_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-2]] + [city_onehot.columns[-1]] + list(city_onehot.columns[:-2])
city_onehot = city_onehot[fixed_columns]
city_onehot.head()

,State,City,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Carpet Store,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Baseball Diamond,College Basketball Court,College Bookstore,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Inn,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaz

In [90]:
city_group = city_onehot.groupby(['State','City']).mean().reset_index()

In [91]:
city_group.head()

,State,City,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Carpet Store,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Baseball Diamond,College Basketball Court,College Bookstore,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Inn,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaz

### Drop cities that already have dispensaries 

In [92]:
city_group = city_group[city_group['Marijuana Dispensary'] == 0]
city_group.reset_index(drop=True,inplace=True)
print(city_group.shape)

(99, 387)


In [93]:
def return_most_common_venues(row, num_top_venues=10):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return  row_categories_sorted.index.values[0:num_top_venues]

### Find top 10 most common venue types for each city

In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['State','City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['State'] = city_group['State']
neighborhoods_venues_sorted['City'] = city_group['City']

for row in np.arange(city_group.shape[0]):
    neighborhoods_venues_sorted.iloc[row, 2:] = return_most_common_venues(city_group.iloc[row,1:],10)
neighborhoods_venues_sorted.head()

,State,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALASKA,ANCHORAGE,Coffee Shop,Park,Clothing Store,Seafood Restaurant,Bar,Accessories Store,Steakhouse,Sporting Goods Shop,Cosmetics Shop,Pizza Place
1,CALIFORNIA,ANAHEIM,Mexican Restaurant,Coffee Shop,Ice Cream Shop,Indian Restaurant,Convenience Store,Taco Place,Liquor Store,Burger Joint,Brewery,Southern / Soul Food Restaurant
2,CALIFORNIA,ANTIOCH,Fast Food Restaurant,Pizza Place,Mexican Restaurant,Chinese Restaurant,Racetrack,Paintball Field,Bakery,Grocery Store,Sports Bar,Burger Joint
3,CALIFORNIA,BAKERSFIELD,Mexican Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,Bar,Chinese Restaurant,Italian Restaurant,Steakhouse,Breakfast Spot,General Entertainment
4,CALIFORNIA,BERKELEY,Chinese Restaurant,Japanese Restaurant,New American Restaurant,Thai Restaurant,French Restaurant,Café,Yoga Studio,Ice Cream Shop,Coffee Shop,Pizza Place


### Performing K-Means Clustering using 5 clusters

I used SciKit Learn K-Means Clustering unsupervised learning in order to group different cities based on their top 10 most common venues.  Once the clustering was performed, I reviewed the venues in the cluster to classify the clusters:  
  
  cluster 0: Shops
  cluster 1: 

In [95]:
k_means = KMeans(init = "k-means++", n_clusters = 5, n_init = 12)

In [96]:
city_group = city_group.merge(city2,on=['City','State'],how="left")

In [111]:
city_group.drop(52,inplace=True)

In [112]:
# set number of clusters
kclusters = 5
from sklearn.preprocessing import StandardScaler

city_clusters = city_group.drop(['State','City','Latitude','Longitude'], axis=1)
city_fit = StandardScaler().fit_transform(city_clusters)

In [113]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_fit)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 4, 1, 1, 1, 1, 1], dtype=int32)

In [115]:
kmeans.labels_.shape

(99,)

In [103]:
city2.reset_index(drop=True,inplace=True)

In [108]:
len(neighborhoods_venues_sorted)

99

In [116]:
# add clustering labels
neighborhoods_venues_sorted['Cluster']=kmeans.labels_

In [117]:
city_full = city2.merge(neighborhoods_venues_sorted, on=['State','City'], how='inner')

In [118]:
city_full.dropna(inplace=True, axis=0)

In [119]:
city_full['Cluster'] = city_full['Cluster'].astype(int)

In [120]:
city_full.head()

,State,City,Tax,Crime Index,CLI,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
0,ALASKA,ANCHORAGE,0.0000,0.026024,94.99,61.216313,-149.894852,Coffee Shop,Park,Clothing Store,Seafood Restaurant,Bar,Accessories Store,Steakhouse,Sporting Goods Shop,Cosmetics Shop,Pizza Place,1
1,CALIFORNIA,ANAHEIM,0.0725,0.013098,0.00,33.834752,-117.911732,Mexican Restaurant,Coffee Shop,Ice Cream Shop,Indian Restaurant,Convenience Store,Taco Place,Liquor Store,Burger Joint,Brewery,Southern / Soul Food Restaurant,1
2,CALIFORNIA,ANTIOCH,0.0725,0.016686,0.00,38.004921,-121.805789,Fast Food Restaurant,Pizza Place,Mexican Restaurant,Chinese Restaurant,Racetrack,Paintball Field,Bakery,Grocery Store,Sports Bar,Burger Joint,1
3,CALIFORNIA,BAKERSFIELD,0.0725,0.019861,69.20,35.373871,-119.019464,Mexican Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,Bar,Chinese Restaurant,Italian Restaurant,Steakhouse,Breakfast Spot,General Entertainment,1
4,CALIFORNIA,BERKELEY,0.0725,0.023042,87.82,37.870839,-122.272864,Chinese Restaurant,Japanese Restaurant,New American Restaurant,Thai Restaurant,French Restaurant,Café,Yoga Studio,Ice Cream Shop,Coffee Shop,Pizza Place,4


In [320]:
city_full.shape

(102, 18)

In [121]:
# create map
map_clusters = folium.Map(location=[39.83, -98.58], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_full['Latitude'], city_full['Longitude'], city_full['City'], city_full['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Review Clusters

#### Cluster 0 

In [122]:
city_full.loc[city_full['Cluster'] == 0, city_full.columns[[1] + list(range(2, city_full.shape[1]))]]

,City,Tax,Crime Index,CLI,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
49,SAN BERNARDINO,0.0725,0.018655,0.0,34.108345,-117.289765,Convenience Store,Clothing Store,Discount Store,Fast Food Restaurant,Grocery Store,Pizza Place,Nightclub,Shoe Store,Department Store,Mexican Restaurant,0


#### Cluster 1

In [123]:
city_full.loc[city_full['Cluster'] == 1, city_full.columns[[1] + list(range(2, city_full.shape[1]))]]

,City,Tax,Crime Index,CLI,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
0,ANCHORAGE,0.0000,0.026024,94.99,61.216313,-149.894852,Coffee Shop,Park,Clothing Store,Seafood Restaurant,Bar,Accessories Store,Steakhouse,Sporting Goods Shop,Cosmetics Shop,Pizza Place,1
1,ANAHEIM,0.0725,0.013098,0.00,33.834752,-117.911732,Mexican Restaurant,Coffee Shop,Ice Cream Shop,Indian Restaurant,Convenience Store,Taco Place,Liquor Store,Burger Joint,Brewery,Southern / Soul Food Restaurant,1
2,ANTIOCH,0.0725,0.016686,0.00,38.004921,-121.805789,Fast Food Restaurant,Pizza Place,Mexican Restaurant,Chinese Restaurant,Racetrack,Paintball Field,Bakery,Grocery Store,Sports Bar,Burger Joint,1
3,BAKERSFIELD,0.0725,0.019861,69.20,35.373871,-119.019464,Mexican Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,Bar,Chinese Restaurant,Italian Restaurant,Steakhouse,Breakfast Spot,General Entertainment,1
5,BURBANK,0.0725,0.014175,0.00,34.181648,-118.325855,Mexican Restaurant,Sandwich Place,American Restaurant,Pizza Place,Burger Joint,Diner,Bakery,Donut Shop,Deli / Bodega,Pet Store,1
6,CARLSBAD,0.0725,0.008791,0.00,33.158093,-117.350597,Beach,Mexican Restaurant,Café,Coffee Shop,Hotel,Breakfast Spot,Pizza Place,Italian Restaurant,American Restaurant,Bar,1
7,CHULA VISTA,0.0725,0.007034,0.00,32.640054,-117.084196,Mexican Restaurant,Grocery Store,Convenience Store,Clothing Store,Taco Place,Sandwich Place,Italian Restaurant,Seafood Restaurant,Cosmetics Shop,Coffee Shop,1
8,CLOVIS,0.0725,0.013983,0.00,36.825228,-119.702919,Mexican Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Burger Joint,Hotel,Ice Cream Shop,American Restaurant,Fast Food Restaurant,Italian Restaurant,1
9,CONCORD,0.0725,0.016411,0.00,37.976852,-122.033562,Mexican Restaurant,Sandwich Place,Discount Store,Japanese Restaurant,Coffee Shop,Café,Chinese Restaurant,Pizza Place,Italian Restaurant,Thai Restaurant,1
10,CORONA,0.0725,0.010682,0.00,33.875295,-117.566445,Mexican Restaurant,Convenience Store,Rental Car Location,Fast Food Restaurant,Indian Restaurant,Sandwich Place,Diner,Discount Store,Sushi Restaurant,Furniture / Home Store,1


#### Cluster 2

In [124]:
city_full.loc[city_full['Cluster'] == 2, city_full.columns[[1] + list(range(2, city_full.shape[1]))]]

,City,Tax,Crime Index,CLI,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
69,CENTENNIAL,0.029,0.008221,0.0,39.568064,-104.977831,Fast Food Restaurant,Sandwich Place,Grocery Store,Pizza Place,American Restaurant,Mexican Restaurant,Gym / Fitness Center,BBQ Joint,Salon / Barbershop,Spa,2


#### Cluster 3

This is our ideal cluster. Active nightlife, pubs, breweries and trendy areas and activities.  

In [125]:
cluster3 = city_full.loc[city_full['Cluster'] == 3, city_full.columns[[1] + list(range(2, city_full.shape[1]))]]
cluster3

,City,Tax,Crime Index,CLI,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
31,LOS ANGELES,0.0725,0.012505,82.30,34.053691,-118.242767,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Plaza,Mexican Restaurant,Ramen Restaurant,Ice Cream Shop,Bookstore,Bar,Mediterranean Restaurant,3
39,ORANGE,0.0725,0.009155,0.00,33.750038,-117.870493,Mexican Restaurant,Convenience Store,Bar,Restaurant,Fast Food Restaurant,Coffee Shop,Pizza Place,Sandwich Place,American Restaurant,Diner,3
42,PASADENA,0.0725,0.010903,0.00,34.147645,-118.144478,American Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Bar,Steakhouse,Bakery,Cosmetics Shop,Pub,Beer Garden,3
50,SAN DIEGO,0.0725,0.009250,77.01,32.717421,-117.162771,Hotel,Mexican Restaurant,Italian Restaurant,Café,Bar,American Restaurant,Coffee Shop,Seafood Restaurant,New American Restaurant,Burger Joint,3
51,SAN JOSE,0.0725,0.012075,82.87,37.336191,-121.890583,Mexican Restaurant,Coffee Shop,Cocktail Bar,Bar,Sushi Restaurant,Sandwich Place,Theater,Pizza Place,Pub,Ice Cream Shop,3
53,SANTA ANA,0.0725,0.011227,0.00,33.749495,-117.873221,Mexican Restaurant,Fast Food Restaurant,Bar,Convenience Store,Restaurant,Pharmacy,Sandwich Place,Pizza Place,American Restaurant,Coffee Shop,3
75,BOSTON,0.0625,0.009736,87.05,42.360253,-71.058291,Italian Restaurant,Seafood Restaurant,Historic Site,Park,Coffee Shop,Pizza Place,Sandwich Place,Market,Bakery,Hotel,3
76,CAMBRIDGE,0.0625,0.008119,82.96,42.375100,-71.105616,New American Restaurant,Coffee Shop,Pub,Café,Gastropub,Brewery,Vegetarian / Vegan Restaurant,Portuguese Restaurant,Ice Cream Shop,Tapas Restaurant,3
82,DETROIT,0.0600,0.022378,70.20,42.331551,-83.046640,Coffee Shop,Bar,American Restaurant,Restaurant,Diner,Park,Lounge,Steakhouse,Burger Joint,Hotel,3
89,RENO,0.0685,0.014195,62.88,39.529270,-119.813674,Bar,Pub,Mexican Restaurant,Brewery,Coffee Shop,Café,Hotel,Breakfast Spot,Steakhouse,Pizza Place,3


#### Cluster 4

In [126]:
city_full.loc[city_full['Cluster'] == 4, city_full.columns[[1] + list(range(2, city_full.shape[1]))]]

,City,Tax,Crime Index,CLI,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
4,BERKELEY,0.0725,0.023042,87.82,37.870839,-122.272864,Chinese Restaurant,Japanese Restaurant,New American Restaurant,Thai Restaurant,French Restaurant,Café,Yoga Studio,Ice Cream Shop,Coffee Shop,Pizza Place,4


In [127]:
# create map
map_clusters = folium.Map(location=[39.83, -98.58], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cluster3['Latitude'], cluster3['Longitude'], cluster3['City'],cluster3['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [135]:
sort3 = cluster3[cluster3['CLI']!=0]

In [148]:
lowtax = sort3.sort_values('Tax').head(6)
lowtax

,City,Tax,Crime Index,CLI,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
82,DETROIT,0.0600,0.022378,70.20,42.331551,-83.046640,Coffee Shop,Bar,American Restaurant,Restaurant,Diner,Park,Lounge,Steakhouse,Burger Joint,Hotel,3
75,BOSTON,0.0625,0.009736,87.05,42.360253,-71.058291,Italian Restaurant,Seafood Restaurant,Historic Site,Park,Coffee Shop,Pizza Place,Sandwich Place,Market,Bakery,Hotel,3
76,CAMBRIDGE,0.0625,0.008119,82.96,42.375100,-71.105616,New American Restaurant,Coffee Shop,Pub,Café,Gastropub,Brewery,Vegetarian / Vegan Restaurant,Portuguese Restaurant,Ice Cream Shop,Tapas Restaurant,3
97,SEATTLE,0.0650,0.025904,88.04,47.603832,-122.330062,Coffee Shop,Cocktail Bar,Seafood Restaurant,Vietnamese Restaurant,American Restaurant,Sushi Restaurant,Italian Restaurant,Breakfast Spot,Hotel,Art Museum,3
89,RENO,0.0685,0.014195,62.88,39.529270,-119.813674,Bar,Pub,Mexican Restaurant,Brewery,Coffee Shop,Café,Hotel,Breakfast Spot,Steakhouse,Pizza Place,3
31,LOS ANGELES,0.0725,0.012505,82.30,34.053691,-118.242767,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Plaza,Mexican Restaurant,Ramen Restaurant,Ice Cream Shop,Bookstore,Bar,Mediterranean Restaurant,3


In [149]:
safe = sort3.sort_values('Crime Index').head(6)
safe

,City,Tax,Crime Index,CLI,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
76,CAMBRIDGE,0.0625,0.008119,82.96,42.375100,-71.105616,New American Restaurant,Coffee Shop,Pub,Café,Gastropub,Brewery,Vegetarian / Vegan Restaurant,Portuguese Restaurant,Ice Cream Shop,Tapas Restaurant,3
50,SAN DIEGO,0.0725,0.009250,77.01,32.717421,-117.162771,Hotel,Mexican Restaurant,Italian Restaurant,Café,Bar,American Restaurant,Coffee Shop,Seafood Restaurant,New American Restaurant,Burger Joint,3
75,BOSTON,0.0625,0.009736,87.05,42.360253,-71.058291,Italian Restaurant,Seafood Restaurant,Historic Site,Park,Coffee Shop,Pizza Place,Sandwich Place,Market,Bakery,Hotel,3
51,SAN JOSE,0.0725,0.012075,82.87,37.336191,-121.890583,Mexican Restaurant,Coffee Shop,Cocktail Bar,Bar,Sushi Restaurant,Sandwich Place,Theater,Pizza Place,Pub,Ice Cream Shop,3
31,LOS ANGELES,0.0725,0.012505,82.30,34.053691,-118.242767,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Plaza,Mexican Restaurant,Ramen Restaurant,Ice Cream Shop,Bookstore,Bar,Mediterranean Restaurant,3
89,RENO,0.0685,0.014195,62.88,39.529270,-119.813674,Bar,Pub,Mexican Restaurant,Brewery,Coffee Shop,Café,Hotel,Breakfast Spot,Steakhouse,Pizza Place,3


In [150]:
lowcol = sort3.sort_values('CLI').head(6)
lowcol

,City,Tax,Crime Index,CLI,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
89,RENO,0.0685,0.014195,62.88,39.529270,-119.813674,Bar,Pub,Mexican Restaurant,Brewery,Coffee Shop,Café,Hotel,Breakfast Spot,Steakhouse,Pizza Place,3
82,DETROIT,0.0600,0.022378,70.20,42.331551,-83.046640,Coffee Shop,Bar,American Restaurant,Restaurant,Diner,Park,Lounge,Steakhouse,Burger Joint,Hotel,3
50,SAN DIEGO,0.0725,0.009250,77.01,32.717421,-117.162771,Hotel,Mexican Restaurant,Italian Restaurant,Café,Bar,American Restaurant,Coffee Shop,Seafood Restaurant,New American Restaurant,Burger Joint,3
31,LOS ANGELES,0.0725,0.012505,82.30,34.053691,-118.242767,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Plaza,Mexican Restaurant,Ramen Restaurant,Ice Cream Shop,Bookstore,Bar,Mediterranean Restaurant,3
51,SAN JOSE,0.0725,0.012075,82.87,37.336191,-121.890583,Mexican Restaurant,Coffee Shop,Cocktail Bar,Bar,Sushi Restaurant,Sandwich Place,Theater,Pizza Place,Pub,Ice Cream Shop,3
76,CAMBRIDGE,0.0625,0.008119,82.96,42.375100,-71.105616,New American Restaurant,Coffee Shop,Pub,Café,Gastropub,Brewery,Vegetarian / Vegan Restaurant,Portuguese Restaurant,Ice Cream Shop,Tapas Restaurant,3


In [157]:
reno = sort3[sort3['City']=='RENO']

## Results

### City Selection

Of the 11 cities in the target cluster, the top 6 safest and 6 lowest state tax and cost of living index were cross-referenced to select Reno, NV followed by Cambridge, MA as the top choices to open a Dispensary in 2019.  Reno was chosen over Cambridge after a tie due to lower cost of living and a more active nightlife, however Cambridge is still a viable choice due to the trendy culture and diversity.

In [163]:
# create map
renomap = folium.Map(location=[39.529,-119.813], zoom_start=10)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(reno['Latitude'], reno['Longitude'], reno['City'], reno['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=200,
        popup=label,
        color='darkgreen',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(renomap)
       
renomap

## Discussion

Upon review of the 5 different cluster datasets, I was able to determine which clusters contain the target demographic and culture that is becoming of a new dispensary such as an active nightlife scene from bars, pubs, liquor stores, activities, lots of restaurants such as fast food and convenience stores.  The recommendation would be to focus investment on opening a dispensary in Reno, NV followed by Cambridge, MA as a backup option or expansion location.  These cities are both highly populated, very trendy, have an active nightlife, are relatively safe, and have a decent state sales tax compared to other cities assessed.  

## Conclusion

Recreational marijuana is a rapidly materializing industry, estimated to surpass $16 Billion in 2019 and exponentially growing.  With the influx of new users and change of legal status, there is a huge demand for recreational marijuana with very limited supply.  This is the prime chance to seize this opportunity and open a new legal marijuana supply chain in the United States.  Based on the data reviewed and analyzed, Reno, Nevada is the top city in the US to open the next Marijuana Dispensary based on its population, safety, cost of living index and overall culture.  